In [53]:
import os
import numpy as np
from cv2 import cv2
from tqdm.auto import tqdm


def get_img_contours(img_path):
    img_names = os.listdir(img_path)
    output = {}

    for img_name in tqdm(img_names):
        image = cv2.imread(img_path + img_name)

        # Resize the image - change width to 500
        newwidth = 500

        # RGB to Gray scale conversion
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Noise removal with iterative bilateral filter(removes noise while preserving edges)
        d, sigmaColor, sigmaSpace = 11, 17, 17
        filtered_img = cv2.bilateralFilter(gray, d, sigmaColor, sigmaSpace)

        # Find Edges of the grayscale image
        lower, upper = 15, 25
        edged = cv2.Canny(filtered_img, lower, upper)

        # Find contours based on Edges
        cnts, hir = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]
        NumberPlateCnt = None

        # loop over our contours to find the best possible approximate contour of number plate
        count = 0
        for c in cnts:
            peri = cv2.arcLength(c, True)

            epsilon = 0.01 * peri
            approx = cv2.approxPolyDP(c, epsilon, True)

            if len(approx) == 4:  # Select the contour with 4 corners
                # print(approx)
                NumberPlateCnt = approx  # This is our approx Number Plate Contour
                break

        if NumberPlateCnt is not None and cv2.contourArea(NumberPlateCnt) > 1250:
            output[img_name] = NumberPlateCnt
            # print(img_name, NumberPlateCnt)
        else:
            # print("LP is not detected.")
            pass

    return output

In [54]:
origin_img_path = 'data/plates/train/a/'
imgs = get_img_contours(origin_img_path)

  0%|          | 0/80000 [00:00<?, ?it/s]

In [55]:
for img_name, points in tqdm(imgs.items()):
    # Read a image
    I = cv2.imread(origin_img_path + img_name)
    points = points.tolist()

    # Define the polygon coordinates to use or the crop
    polygon = [[p[0] for p in points]]

    # First find the minX minY maxX and maxY of the polygon
    minX = I.shape[1]
    maxX = -1
    minY = I.shape[0]
    maxY = -1
    for point in polygon[0]:

        x = point[0]
        y = point[1]

        if x < minX:
            minX = x
        if x > maxX:
            maxX = x
        if y < minY:
            minY = y
        if y > maxY:
            maxY = y

    # Go over the points in the image if thay are out side of the emclosing rectangle put zero
    # if not check if thay are inside the polygon or not
    cropedImage = np.zeros_like(I)
    for y in range(0,I.shape[0]):
        for x in range(0, I.shape[1]):

            if x < minX or x > maxX or y < minY or y > maxY:
                continue

            if cv2.pointPolygonTest(np.asarray(polygon),(x,y),False) >= 0:
                cropedImage[y, x, 0] = I[y, x, 0]
                cropedImage[y, x, 1] = I[y, x, 1]
                cropedImage[y, x, 2] = I[y, x, 2]

    # Now we can crop again just the envloping rectangle
    # finalImage = cropedImage[minY:maxY,minX:maxX]
    # finalImage

    cv2.imwrite('data/plates/cropped/' + img_name, cropedImage)

  0%|          | 0/17331 [00:00<?, ?it/s]

In [56]:
crops = os.listdir('data/plates/cropped/')
len(crops)

17428

In [74]:
import shutil

for crop in tqdm(crops):
    im = cv2.imread('data/plates/cropped/' + crop)
    dst0 = cv2.bilateralFilter(im, 9, 20, 20)
    dst1 = cv2.GaussianBlur(dst0, (3,3), 5,5)
    canny = cv2.Canny(dst1, 150, 150)
    cv2.imwrite('data/plates2/train/b/' + crop, canny)
    shutil.copy('data/plates/train/a/' + crop, 'data/plates2/train/a/' + crop)

    # cv2.imshow('', im)
    # cv2.imshow('dst0', dst0)
    # cv2.imshow('dst1', dst1)
    # cv2.imshow('canny', canny)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # break

  0%|          | 0/17428 [00:00<?, ?it/s]